In [2]:
from components.fetch_data import FetchData
from components.google_credentials import *
from components.initialize_credentials import init_credentials
import pandas as pd

In [3]:
def main(sheet_name: str) -> None:
    #GOOGLE BUCKET STORAGE CREDENTIALS +++++++
    bucket_name = 'hkecommercedata'
    folder_name = 'fb-ads'

    # Create a Credentials object from the access token and refresh token
    # and authorize the gspread client with the credential objects
    client = init_credentials(ACCESS_TOKEN, REFRESH_TOKEN, CLIENT_ID, CLIENT_SECRET, TOKEN_URI)

    # Open the Google Sheet by URL and get the data as a pandas DataFrame
    sheet = client.open_by_url(SHEET_URL)
    worksheet = sheet.worksheet(sheet_name)
    data = worksheet.get_all_values()

    # delete duplicates and drop "Profile id" with NaN values and retains the first value.
    df = pd.DataFrame(data[1:], columns=data[0]).dropna(subset=["Profile ID"]).drop_duplicates(subset=["Profile ID"])
    df["Added Date"] = pd.to_datetime(df["Added Date"])
    df.sort_values(by = ["Profile"], inplace = True)

    ## additional conditions that delete rows with empty string or with #N/A str values
    df = df[~df["Profile ID"].isin(["", "#N/A"])] 

    # call the class
    # this will directly call upload the file to google storage
    instance = FetchData(df)
    instance.process_data_now()

    instance.upload_to_google_storage(
        bucket_name=bucket_name,
        folder_name=folder_name,
        filename="fbadsLatest3.json"
    )
    
    instance.upload_to_local("fbadslatest3.json") 

main("Sheet1")

File fbadsLatest3.json uploaded to Google Cloud Storage.
Upload successfully on local drive: path is in json folder.


In [ ]:
def main(sheet_name: str) -> None:
    bucket_name = 'hkecommercedata'
    folder_name = 'fb-ads'

    client = init_credentials(ACCESS_TOKEN, REFRESH_TOKEN, CLIENT_ID, CLIENT_SECRET, TOKEN_URI)

    sheet = client.open_by_url(SHEET_URL)
    worksheet = sheet.worksheet(sheet_name)
    data = worksheet.get_all_values()

    df = pd.DataFrame(data[1:], columns=data[0]).dropna(subset=["Profile ID"]).drop_duplicates(subset=["Profile ID"])
    df["Added Date"] = pd.to_datetime(df["Added Date"])
    df.sort_values(by = ["Profile"], inplace = True)

    df = df[~df["Profile ID"].isin(["", "#N/A"])] 

    instance = FetchData(df)
    instance.process_data_now()

    instance.upload_to_google_storage(
        bucket_name=bucket_name,
        folder_name=folder_name,
        filename="fbadsLatest3.json"
    )
    
    instance.upload_to_local("fbadslatest3.json") 

main("Sheet1")